### Гипотеза
Нужно проверить:
- Зависит ли вероятность оплаты от выбранного пользователем уровня сложности бесплатных тренировок?
- Существует ли разница во времени между пользователями с разным уровнем сложности и их первой оплатой?

### Проверка гипотезы
Проверку будем производить на основе данных пользователей, которые зарегистрировались в 2017 году (с 1 января по 31 декабря 2017 года включительно).

In [1]:
import pandas as pd #импортируем библиотеки
import psycopg2
import psycopg2.extras 
import numpy as np

Получаем данные из базы данных

In [2]:
def getEventsData():
    query = '''SELECT e.* FROM case8.events e
    WHERE e.user_id in
    (SELECT DISTINCT(user_id) 
    FROM case8.events 
    WHERE event_type = 'registration' 
    AND start_time >= '2017-01-01'
    AND start_time < '2018-01-01')
    '''.format()
    conn = psycopg2.connect("dbname='' user='' host='' password='' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

def getPurchaseData():
    query = '''SELECT p.* FROM case8.purchase p
    WHERE p.user_id in
    (SELECT DISTINCT(user_id) 
    FROM case8.events 
    WHERE event_type = 'registration' 
    AND start_time >= '2017-01-01'
    AND start_time < '2018-01-01')
    '''.format()
    conn = psycopg2.connect("dbname='' user='' host='' password='' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

events_df = pd.DataFrame(getEventsData())
purchase_df = pd.DataFrame(getPurchaseData())

Произведем обзор данных и преобразование данных

In [3]:
events_df['start_time'] = pd.to_datetime(events_df['start_time'])
purchase_df['start_time'] = pd.to_datetime(purchase_df['start_time'])

In [4]:
purchase_df['event_type'] = 'purchase' #объединим датафрейм с событиями вместе с датафреймом по оплатам

In [5]:
events_df = events_df.rename(columns={'id':'event_id'}) #переименуем колонки
purchase_df = purchase_df.rename(columns={'id':'purchase_id'})

In [6]:
total_events_df = pd.concat([events_df,purchase_df],sort=False) #запишем объединенный датафрейм в переменную total_events_df

Выделим группы пользователей по уровню сложности.

In [7]:
total_events_df['selected_level'].unique() #какие уникальные уровни сложности есть в датафрейме

array([None, 'medium', 'hard', 'easy', nan], dtype=object)

In [8]:
users_with_easy_level = total_events_df[total_events_df['selected_level'] == 'easy']['user_id'].unique()

In [9]:
users_with_medium_level = total_events_df[total_events_df['selected_level'] == 'medium']['user_id'].unique()

In [10]:
users_with_hard_level = total_events_df[total_events_df['selected_level'] == 'hard']['user_id'].unique()

Рассчитаем для каждой группы процент оплат

In [11]:
set_users_with_easy_level = set(users_with_easy_level) #уникальные пользователи, выбравшие уровень easy
set_users_with_medium_level = set(users_with_medium_level) #уникальные пользователи, выбравшие уровень medium
set_users_with_hard_level = set(users_with_hard_level) #уникальные пользователи, выбравшие уровень hard

purchase_df_easy = purchase_df[purchase_df['user_id'].isin(set_users_with_easy_level)] #процент оплат для уровня easy
percent_of_purchase_easy = purchase_df_easy['user_id'].nunique()/len(set_users_with_easy_level)
print ('Процент оплативших пользователей, выбравших уровень сложности easy: {:.2%}'.format(percent_of_purchase_easy))

purchase_df_medium = purchase_df[purchase_df['user_id'].isin(set_users_with_medium_level)] #процент оплат для уровня medium
percent_of_purchase_medium = purchase_df_medium['user_id'].nunique()/len(set_users_with_medium_level)
print ('Процент оплативших пользователей, выбравших уровень сложности medium: {:.2%}'.format(percent_of_purchase_medium))

purchase_df_hard = purchase_df[purchase_df['user_id'].isin(set_users_with_hard_level)] #процент оплат для уровня hard
percent_of_purchase_hard = purchase_df_hard['user_id'].nunique()/len(set_users_with_hard_level)
print ('Процент оплативших пользователей, выбравших уровень сложности hard: {:.2%}'.format(percent_of_purchase_hard))

Процент оплативших пользователей, выбравших уровень сложности easy: 7.72%
Процент оплативших пользователей, выбравших уровень сложности medium: 20.86%
Процент оплативших пользователей, выбравших уровень сложности hard: 35.39%


Для каждой группы подсчитаем среднее время между событиями оплаты и событием выбора уровня сложности

In [12]:
#Группа easy:
level_choice_df_easy = total_events_df[(total_events_df['event_type'] == 'level_choice') 
                                       & (total_events_df['selected_level'] == 'easy')]
level_choice_df_easy = level_choice_df_easy[['user_id','start_time']].rename(columns={'start_time':'level_choice_time'})
purchase_df_2_easy = purchase_df_easy[['user_id','start_time']].rename(columns={'start_time':'purchase_time'})
merged_df_easy = purchase_df_2_easy.merge(level_choice_df_easy,on='user_id',how='inner')
merged_df_easy['timedelta'] = merged_df_easy['purchase_time'] - merged_df_easy['level_choice_time']
mean_time1 = merged_df_easy['timedelta'].mean()
print('Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности easy: {0}'.format(mean_time1))
print('Показатели:')
print(merged_df_easy['timedelta'].describe())

#Группа medium:
level_choice_df_medium = total_events_df[(total_events_df['event_type'] == 'level_choice') 
                                       & (total_events_df['selected_level'] == 'medium')]
level_choice_df_medium = level_choice_df_medium[['user_id','start_time']].rename(columns={'start_time':'level_choice_time'})
purchase_df_2_medium = purchase_df_medium[['user_id','start_time']].rename(columns={'start_time':'purchase_time'})
merged_df_medium = purchase_df_2_medium.merge(level_choice_df_medium,on='user_id',how='inner')
merged_df_medium['timedelta'] = merged_df_medium['purchase_time'] - merged_df_medium['level_choice_time']
mean_time2 = merged_df_medium['timedelta'].mean()
print('Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности medium: {0}'.format(mean_time2))
print('Показатели:')
print(merged_df_medium['timedelta'].describe())

#Группа hard:
level_choice_df_hard = total_events_df[(total_events_df['event_type'] == 'level_choice') 
                                       & (total_events_df['selected_level'] == 'hard')]
level_choice_df_hard = level_choice_df_hard[['user_id','start_time']].rename(columns={'start_time':'level_choice_time'})
purchase_df_2_hard = purchase_df_hard[['user_id','start_time']].rename(columns={'start_time':'purchase_time'})
merged_df_hard = purchase_df_2_hard.merge(level_choice_df_hard,on='user_id',how='inner')
merged_df_hard['timedelta'] = merged_df_hard['purchase_time'] - merged_df_hard['level_choice_time']
mean_time3 = merged_df_hard['timedelta'].mean()
print('Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности hard: {0}'.format(mean_time3))
print('Показатели:')
print(merged_df_hard['timedelta'].describe())

Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности easy: 3 days 14:58:52.941798
Показатели:
count                       189
mean     3 days 14:58:52.941798
std      2 days 07:06:35.644097
min             0 days 00:49:20
25%             1 days 17:18:56
50%             3 days 06:03:50
75%             5 days 06:58:18
max            10 days 18:35:09
Name: timedelta, dtype: object
Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности medium: 3 days 23:14:13.165118
Показатели:
count                       969
mean     3 days 23:14:13.165118
std      2 days 06:18:57.618467
min             0 days 04:18:12
25%             2 days 01:20:07
50%             3 days 19:53:19
75%             5 days 16:07:19
max            10 days 13:51:01
Name: timedelta, dtype: object
Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности hard: 3 days 07:20:41.420814
Показат

### Выводы

*Зависит ли вероятность оплаты от выбранного пользователем уровня сложности бесплатных тренировок?*

**Процент оплативших пользователей**, выбравших уровень сложности **easy: 7.72%**
**Процент оплативших пользователей**, выбравших уровень сложности **medium: 20.86%**
**Процент оплативших пользователей**, выбравших уровень сложности **hard: 35.39%**

Таким образом, существует зависимость между выбранным уровнем сложности бесплатных тренировок и процентом пользователей, которые оплатили тренировки. Пользователи, выбирающие более сложный уровень тренировки, чаще покупают платные тренировки.

*Существует ли разница во времени между пользователями с разным уровнем сложности и их первой оплатой?*

Наибольшее среднее время между выбором уровня сложности и первой оплатой принадлежит пользователям, которые выбрали уровень сложности **medium**. Для них среднее время **составляет 3 дня 23 часа**.
Для пользователей, выбравших уровень сложности **easy**, среднее время **составляет 3 дня 14 часов**.
Для пользователей, выбравших уровень сложности **hard**, среднее время **составляет 3 дня 7 часов**. 
Таким образом, пользователи, выбравшие уровень сложности hard тратят меньше всего времени на совершение оплаты.